# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

import warnings 
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.tail()

,calls,minutes,messages,mb_used,is_ultra
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0
3213,80.0,566.09,6.0,29480.52,1


In [5]:
df.shape

(3214, 5)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [7]:
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [8]:
df.duplicated().sum()

0

In [9]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [10]:
df['messages'] = df['messages'].astype('Int64')
df['calls'] = df['calls'].astype('Int64')

## Разбейте данные на выборки

In [11]:
X = df.drop('is_ultra', axis=1)
y = df['is_ultra']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, random_state=42)

In [13]:
print('Размерность обучающей выборки',X_train.shape)
print('Размерность тестовой выборки',X_test.shape)

Размерность обучающей выборки (2571, 4)
Размерность тестовой выборки (643, 4)


In [14]:
X_train

,calls,minutes,messages,mb_used
2620,20,104.02,8,15873.88
1042,97,760.53,77,20246.27
2268,10,78.67,8,4986.91
1078,91,604.39,19,25152.82
420,67,505.20,25,16225.71
...,...,...,...,...
1095,62,454.02,35,15018.46
1130,69,465.96,12,14982.27
1294,40,280.44,2,13934.54
860,72,410.23,68,16006.55


Сделаем стандартизацию.

In [15]:
numeric = ['calls', 'minutes', 'messages', 'mb_used']

scaler = StandardScaler()
scaler.fit(X_train[numeric])

X_train[numeric] = scaler.transform(X_train[numeric])
X_test[numeric] = scaler.transform(X_test[numeric])

## Исследуйте модели

<b>Дерево принятия решений</b>

In [16]:
model_dt = DecisionTreeClassifier(random_state=42)
param_dt = {
    'max_depth' : [8,12,16,20,24,28],
    'min_samples_split' : [2,3,4,5,6],
    'min_samples_leaf' : [1,2,3]
}

In [17]:
grid_dt = GridSearchCV(model_dt, param_dt, cv=3, n_jobs=-1, scoring='accuracy')
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [8, 12, 16, 20, 24, 28],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 3, 4, 5, 6]},
             scoring='accuracy')

In [18]:
grid_dt.best_score_

0.7895760404511863

In [19]:
grid_dt.best_params_

{'max_depth': 8, 'min_samples_leaf': 3, 'min_samples_split': 2}

<b>Случайный лес</b>

In [20]:
model_lr = RandomForestClassifier(random_state=42)
param_lr = {
    'max_depth' : [8,12,16,20,24,28],
    'min_samples_split' : [2,3,4,5,6],
    'min_samples_leaf' : [1,2,3]
}

In [21]:
grid_lr = GridSearchCV(model_lr, param_lr, cv=3, n_jobs=-1, scoring='accuracy')
grid_lr.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [8, 12, 16, 20, 24, 28],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 3, 4, 5, 6]},
             scoring='accuracy')

In [22]:
grid_lr.best_score_

0.8043562816024893

In [23]:
grid_lr.best_params_

{'max_depth': 8, 'min_samples_leaf': 3, 'min_samples_split': 2}

<b>Логистическая регрессия</b>

In [24]:
model_logistic = LogisticRegression(random_state=42)
param_logistic = {
    'C' : [0.01,0.1,1,10,100],
    'class_weight' : ['balanced', None]
}

In [25]:
grid_logistic = GridSearchCV(model_logistic, param_logistic, cv=3, n_jobs=-1, scoring='accuracy')
grid_logistic.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=LogisticRegression(random_state=42), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'class_weight': ['balanced', None]},
             scoring='accuracy')

In [26]:
grid_logistic.best_score_

0.7421236872812136

In [27]:
grid_logistic.best_params_

{'C': 0.1, 'class_weight': None}

## Проверьте модель на тестовой выборке

Лучше всех справился случайный лес. Проверим его на тестовой выборке.

In [28]:
pred = grid_lr.best_estimator_.predict(X_test)
accuracy = accuracy_score(pred, y_test)
print('Качество на тестовой выборке:', accuracy)

Качество на тестовой выборке: 0.8149300155520995


## (бонус) Проверьте модели на адекватность

Проверка на вменяемость предполагает сравнение с наивной моделью, которая, например, всегда рекомендует самый частый класс <br><br>

ML-модель должна продемонстрировать преимущество над Dummy-моделью (реализующей наивный подход) <br><br>

В scikit-learn уже есть готовые реализации, например для задачи классификации: https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html

In [29]:
dummy_clf = DummyClassifier(strategy="uniform", random_state = 42)
dummy_clf.fit(X_train, y_train)
dummy_accuracy = dummy_clf.score(X_test, y_test)
print('Качество наивного прогноза:', dummy_accuracy)
print('Качество случайного леса:', accuracy)

Качество наивного прогноза: 0.49300155520995337
Качество случайного леса: 0.8149300155520995


In [30]:
dummy_clf = DummyClassifier(strategy="stratified", random_state = 42)
dummy_clf.fit(X_train, y_train)
dummy_accuracy = dummy_clf.score(X_test, y_test)
print('Качество наивного прогноза:', dummy_accuracy)
print('Качество логистической регрессии:', accuracy)

Качество наивного прогноза: 0.5629860031104199
Качество логистической регрессии: 0.8149300155520995


In [31]:
dummy_clf = DummyClassifier(strategy="most_frequent", random_state = 42)
dummy_clf.fit(X_train, y_train)
dummy_accuracy = dummy_clf.score(X_test, y_test)
print('Качество наивного прогноза:', dummy_accuracy)
print('Качество логистической регрессии:', accuracy)

Качество наивного прогноза: 0.7076205287713841
Качество логистической регрессии: 0.8149300155520995


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
